In [1]:
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from mpl_toolkits.mplot3d import Axes3D
import gymnasium
from run.train import configuration
import json
from copy import copy, deepcopy
import panda_gym
import pybullet
from sb3_contrib import TQC

C:\Users\eclip\GitHub\panda-gym\venv\Lib\site-packages\gymnasium-0.28.1-py3.11.egg\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment PandaReachAO-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
human=True
reward_type="sparse"
evaluation_scenario="library2"
n_substeps=20
prior_orientation=""
max_ep_steps = 100
goal_condition="reach"


In [3]:
#register envs
panda_gym.register_reach_ao(max_ep_steps)

In [4]:
# get rendered env
env = gymnasium.make(configuration["env_name"], render=True, control_type="jsd",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition="halt",
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=True, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)

In [5]:
with open("scenario_goals", "r") as file:
    scenario_goals = json.load(file)
goals_to_achieve = copy(scenario_goals[evaluation_scenario])

In [6]:
# get work env
work_env = gymnasium.make(configuration["env_name"], render=False, control_type="jsd",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition=goal_condition,
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=True, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)


results, metrics = evaluate.evaluate_ensemble(["prior"], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="", goals_to_achieve=goals_to_achieve,
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation)
ee_pos = metrics["end_effector_positions"]

In [7]:
model_name = "silver-star-416"
model = TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=env,
                 custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),
                                                                      dtype=np.float32)})  # for some reason it won't read action space sometimes

results, metrics = evaluate.evaluate_ensemble([model], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="", goals_to_achieve=goals_to_achieve,
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


library2: 100%|██████████| 100/100 [00:54<00:00,  1.84it/s]


In [8]:
# change visual shape of robot

for trajectory in ee_pos:
    color = 1.0
    traj = deepcopy(trajectory)
    xyz1 = traj.pop()
    while traj:
        xyz2 = traj.pop()
        pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([1-color,color,0]), physicsClientId=0, lifeTime=0, lineWidth=2)
        color = len(traj)/len(trajectory)#1/max_ep_steps
        xyz1 = xyz2

# plot with debug line


# Plotting
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
#
# ax.plot(x, y, z, label='3D trajectory')
# ax.legend()
#
# plt.show()

In [9]:
# pybullet.removeAllUserDebugItems()

In [11]:

pybullet.changeVisualShape(objectUniqueId=0, linkIndex=4, rgbaColor=[1,1,1,1])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=2, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=0, rgbaColor=[1,1,1,0.5])